# 評価手法の検討

In [1]:
# Cell 1: 環境設定とセットアップ
"""
LA-Bench 2025: 実験手順生成タスク
Baseline Implementation for Google Colaboratory
GitHub: https://github.com/lasa-or-jp/la-bench.git
"""

#@title 1. 環境セットアップ { display-mode: "form" }
#@markdown このセルを実行して必要なライブラリをインストールし、リポジトリをクローンします。


import os
from pathlib import Path

# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colaboratory環境を検出しました")
    # 必要なライブラリのインストール
    print("\n📦 必要なライブラリをインストール中...")
    !pip install -q openai pyyaml tqdm pandas

    print("✅ ライブラリのインストール完了")

    # GitHubリポジトリのクローン
    REPO_URL = "https://github.com/lasa-or-jp/la-bench.git"
    REPO_NAME = "la-bench"

    if not os.path.exists(REPO_NAME):
        print(f"\n📥 リポジトリをクローン中: {REPO_URL}")
        !git clone -q {REPO_URL}
        print(f"✅ リポジトリのクローン完了: {REPO_NAME}/")
    else:
        print(f"\n📂 リポジトリは既に存在します: {REPO_NAME}/")
        print("📥 最新版に更新中...")
        !cd {REPO_NAME} && git pull -q
        print("✅ 更新完了")

    # 作業ディレクトリの設定
    WORK_DIR = Path(REPO_NAME)
    os.chdir(WORK_DIR)
    print(f"\n📍 作業ディレクトリ: {os.getcwd()}")

    # ディレクトリ構造の確認
    print("\n📊 プロジェクト構造:")
    !ls -la
except ImportError:
    IN_COLAB = False
    print("⚠️ ローカル環境で実行中です")
    if Path.cwd().name == "notebooks":
        os.chdir(Path.cwd().parent)

⚠️ ローカル環境で実行中です


In [2]:
# Cell 2: OpenAI APIキーの設定
#@title 2. OpenAI API Key設定 { display-mode: "form" }
#@markdown OpenAI APIキーを入力してください。キーは安全に管理されます。


# APIキーの取得方法を選択
use_secrets = True  #@param {type:"boolean"}
#@markdown ☝️ Google Colab Secretsを使用する場合はチェック

if IN_COLAB:
    import getpass
    from google.colab import userdata
    if use_secrets:
        try:
            # Colab Secretsから取得
            API_KEY = userdata.get('OPENAI_API_KEY')
            print("✅ APIキーをSecretsから取得しました")
        except Exception as e:
            print("⚠️ Secretsからの取得に失敗しました")
            print("左側のパネルの🔑アイコンから'OPENAI_API_KEY'を設定してください")
            API_KEY = None
    else:
        # 直接入力
        api_key_input = getpass.getpass("🔑 OpenAI API Keyを入力: ")
        if api_key_input:
            API_KEY = api_key_input
            os.environ['OPENAI_API_KEY'] = API_KEY
            print("✅ APIキーが設定されました")
        else:
            API_KEY = None
            print("⚠️ APIキーが設定されていません（ヒューリスティック手法のみ使用）")
else:
    # ローカル環境の場合
    API_KEY = os.getenv("OPENAI_API_KEY")
    if not API_KEY:
        API_KEY = input("OpenAI API Key: ")

# APIキーの検証
if API_KEY:
    print(f"🔑 APIキー: {'*' * 20}{API_KEY[-4:]}")
else:
    print("⚠️ GPT機能は使用できません")

🔑 APIキー: ********************6tcA


In [3]:
# Cell 3: ライブラリのインポートと設定
#@title 3. ライブラリのインポート { display-mode: "form" }

import json
import yaml
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Set, Any
from pathlib import Path
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# データ処理
import pandas as pd
from dataclasses import dataclass, field

# OpenAI API
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAIライブラリが利用できません")

# プログレスバー (Colab対応)
from tqdm.auto import tqdm

# ログ設定
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("="*60)
print("LA-Bench 2025 Baseline Implementation")
print(f"実行環境: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"OpenAI利用可能: {OPENAI_AVAILABLE and API_KEY is not None}")
print("="*60)

LA-Bench 2025 Baseline Implementation
実行環境: Local
実行時刻: 2025-12-23 14:57:56
OpenAI利用可能: True


In [4]:
# Cell 4: データ構造
#@title 4. データ構造の定義 { display-mode: "form" }

@dataclass
class Step:
    id: int
    text: str

@dataclass
class ReferenceEntry:
    id: int
    text: str

@dataclass
class ExampleInput:
    instruction: str
    mandatory_objects: Set[str] = field(default_factory=set)
    source_protocol_steps: List[Step] = field(default_factory=list)
    expected_final_states: Set[str] = field(default_factory=set)
    references: List[ReferenceEntry] = field(default_factory=list)

@dataclass
class ExampleOutput:
    procedure_steps: List[Step] = field(default_factory=list)

@dataclass
class Measurement:
    specific_criteria: Dict[str, int] = field(default_factory=dict)

@dataclass
class ExampleSample:
    id: str
    input: ExampleInput
    output: ExampleOutput
    measurement: Optional[Measurement] = None

def _to_set(x):
    return set(x) if isinstance(x, (list, set, tuple)) else set()

def _to_list(x):
    return list(x) if isinstance(x, (list, set, tuple)) else (x if isinstance(x, list) else [])

def _to_steps(x) -> List[Step]:
    steps: List[Step] = []
    arr = _to_list(x)
    if not arr:
        return steps
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                sid = int(it.get("id", len(steps) + 1))
            except Exception:
                sid = len(steps) + 1
            steps.append(Step(id=sid, text=str(it.get("text", "")).strip()))
    else:
        for idx, s in enumerate(arr, start=1):
            steps.append(Step(id=idx, text=str(s).strip()))
    return steps

def _to_references(x) -> List[ReferenceEntry]:
    refs: List[ReferenceEntry] = []
    arr = _to_list(x)
    if not arr:
        return refs
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                rid = int(it.get("id", len(refs) + 1))
            except Exception:
                rid = len(refs) + 1
            refs.append(ReferenceEntry(id=rid, text=str(it.get("text", "")).strip()))
    else:
        for idx, ref in enumerate(arr, start=1):
            refs.append(ReferenceEntry(id=idx, text=str(ref).strip()))
    return refs

def parse_sample(obj: Dict[str, Any]) -> ExampleSample:
    sid = obj.get("id") or obj.get("sample_id") or "unknown"
    i = obj.get("input", {})
    o = obj.get("output", {})
    m = obj.get("measurement", {})

    # Measurement.specific_criteria を dict に正規化（list形式も許容）
    sc_raw = m.get("specific_criteria", {})
    sc: Dict[str, int] = {}
    if isinstance(sc_raw, dict):
        for k, v in sc_raw.items():
            try:
                sc[str(k)] = int(v)
            except Exception:
                pass
    elif isinstance(sc_raw, list):
        for it in sc_raw:
            try:
                k = it.get("item")
                v = int(it.get("score", 0))
                if k:
                    sc[str(k)] = v
            except Exception:
                pass

    sample = ExampleSample(
        id=str(sid),
        input=ExampleInput(
            instruction=str(i.get("instruction", "")).strip(),
            mandatory_objects=_to_set(i.get("mandatory_objects", [])),
            source_protocol_steps=_to_steps(i.get("source_protocol_steps", [])),
            expected_final_states=_to_set(i.get("expected_final_states", [])),
            references=_to_references(i.get("references", [])),
        ),
        output=ExampleOutput(
            procedure_steps=_to_steps(o.get("procedure_steps", []))
        ),
        measurement=Measurement(specific_criteria=sc) if sc else None
    )
    return sample

def load_example_jsonl(path: str):
    samples = []
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"JSONL not found: {p}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"⚠️ JSONL parse error: {e}")
            continue
        samples.append(parse_sample(obj))
    return samples

In [5]:
# Cell 5: JSONLローダーの利用
#@title 5. JSONLファイルを読み込む { display-mode: "form" }
#@markdown exampleを使うとき：`data/example/example.jsonl`
#@markdown public_testを使うとき：`data/public_test/public_test.jsonl`

# jsonl_path = 'data/example/example.jsonl'  #@param {type:'string'}
# jsonl_path = 'data/public_test/public_test.jsonl'  #@param {type:'string'}
jsonl_path = 'data/private_test/private_test.jsonl'  #@param {type:'string'}

try:
    samples = load_example_jsonl(jsonl_path)
    print(f'✅ Loaded {len(samples)} samples from {jsonl_path}')
except Exception as e:
    print(f'❌ Load error: {e}')


✅ Loaded 10 samples from data/private_test/private_test.jsonl


In [6]:
# Cell 6: 実験手順の生成（OpenAI, Pydantic構造化）
#@title 6. LLMで Input から Output（procedure_steps）を生成 { display-mode: "form" }

from pydantic import BaseModel, Field

# モデル設定
MODEL_NAME = "gpt-5-mini-2025-08-07" #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
#@markdown gpt-4o-mini, gpt-4o-2024-08-06, あるいはそれ以降のモデルに対応しています。<br>
#@markdown (Structured outputを使用しているため) <br>
#@markdown gpt-5系モデルを使用する場合、temperature=1.0としてください。
TEMPERATURE = 1.0 # @param
#@markdown `build_messages`関数において、LLMの入力を設計しています。

class StepModel(BaseModel):
    id: int = Field(ge=1, description="ステップ番号")
    text: str = Field(description="実験手順の詳細な説明")

class GeneratedOutput(BaseModel):
    procedure_steps: List[StepModel] = Field(
        description="実験手順のリスト",
        min_items=1,
        max_items=50
    )

def build_messages(sample: ExampleSample) -> list[dict]:
    sys = (
        "あなたは生命科学実験の専門家です。以下の Input を読み、"
        "日本語で実行可能な実験手順（procedure_steps）を返してください。"
        "制約: ステップ数は最大50、各ステップは10文以下、idは1から昇順。"
    )
    user_lines = []
    user_lines.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        user_lines.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            user_lines.append(f"- {it}")
    if sample.input.source_protocol_steps:
        user_lines.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            user_lines.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        user_lines.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            user_lines.append(f"- {fs}")
    if sample.input.references:
        user_lines.append("\n【参考文献】")
        for ref in sample.input.references:
            user_lines.append(f"- [{ref.id}] {ref.text}")
    usr = "\n".join(user_lines)
    return [
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
    ]

def generate_outputs(samples: list[ExampleSample]) -> list[dict]:
    client = OpenAI(api_key=API_KEY)
    results: list[dict] = []
    for sm in samples:
        msgs = build_messages(sm)
        try:
            completion = client.chat.completions.parse(
                model=MODEL_NAME,
                messages=msgs,
                temperature=TEMPERATURE,
                response_format=GeneratedOutput,
            )
            parsed: GeneratedOutput = completion.choices[0].message.parsed  # type: ignore
            steps = [
                Step(id=s.id, text=s.text)
                for s in sorted(parsed.procedure_steps, key=lambda x: x.id)
            ][:50]
        except Exception as e:
            print(f"❌ 生成失敗: {sm.id}: {e}")
            steps = []  # no fallback
        results.append({
            "id": sm.id,
            "procedure_steps": [{"id": s.id, "text": s.text} for s in steps],
        })
    print(f"✅ 生成完了: {len(results)} samples")
    return results

# 実行
generated_results = generate_outputs(samples)
if generated_results:
    print(f"例: {generated_results[0]['id']} → {len(generated_results[0]['procedure_steps'])} steps")

# 生成結果を JSONL で保存し、ダウンロードリンクを表示
ts = time.strftime('%Y%m%d_%H%M%S')
out_dir = Path('./outputs/runs')
out_dir.mkdir(parents=True, exist_ok=True)
jsonl_path = out_dir / f'generated_{ts}.jsonl'
with jsonl_path.open('w', encoding='utf-8') as f:
    for rec in generated_results:
        obj = {"id": rec["id"], "output": {"procedure_steps": rec["procedure_steps"]}}
        line = json.dumps(obj, ensure_ascii=False, separators=(",", ":"))
        f.write(line + "\n")
print(f"📄 Saved JSONL: {jsonl_path}")

# ダウンロード（Colab/ローカル双方に対応）
try:
    from google.colab import files as colab_files  # type: ignore
    # ダウンロード確認ダイアログを出して、yならダウンロード
    from google.colab.output import eval_js
    print(f"Download file: {jsonl_path}")
    confirm = eval_js('confirm("生成されたJSONLファイルをダウンロードしますか？")')
    if confirm:
      colab_files.download(str(jsonl_path))
    else:
      print("ダウンロードをスキップしました。")

except Exception:
    from IPython.display import FileLink, display
    display(FileLink(str(jsonl_path.resolve())))

14:58:36 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:59:46 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:01:43 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:03:11 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:04:23 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:05:19 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:06:04 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:06:43 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:07:51 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:08:26 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ 生成完了: 10 samples
例: private_test_1 → 16 steps
📄 Saved JSONL: outputs/runs/generated_20251223_150826.jsonl


/Users/kato/Dropbox/Programs/gensurv/la-bench/outputs/runs/generated_20251223_150826.jsonl

In [7]:
# Cell 7: LLM-as-a-judge 評価（10点満点）
#@title 7. LLM で共通5点 + 個別5点を採点 { display-mode: "form" }

from typing import List, Optional
from pydantic import BaseModel, Field

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("OpenAI SDK v1 が見つかりません。`uv add openai` で追加してください。") from e

JUDGE_MODEL = "gpt-5-mini-2025-08-07"  #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
JUDGE_TEMPERATURE = 1.0

class JudgeOutput(BaseModel):
    general_score: float = Field(ge=0, le=5)
    specific_score: float = Field(ge=0, le=5)
    final_score: float = Field(ge=0, le=10)
    general_reason: str
    specific_matches: List[str] = []
    notes: Optional[str] = None

def build_judge_messages(sample: ExampleSample, steps: List[Step]) -> list[dict]:
    # 評価基準（共通5点 + 個別5点）
    system = (
        "あなたは生命科学実験の専門家であり、公平な採点者です。"
        "以下の基準に従って、与えられた Input に対して【生成された実験手順（Output）】を評価し、"
        "general_score(0-5) と specific_score(0-5) と final_score(0-10) を出力してください。"
        "\n\n[共通採点基準 5点満点]\n"
        "加点(+1ずつ): 1) 実験指示のパラメータ反映, 2) 使用する物品の反映, 3) 元手順の論理反映, 4) 期待される最終状態の達成, 5) 適切な補完。\n"
        "減点: 不自然な日本語/ハルシネーション, 計算ミス, 手順矛盾。\n"
        "上限: 入力手順の丸写し等の過度の安全性が見られる場合、general_score は最大2点に制限。\n\n"
        "[個別採点基準 5点満点]\n"
        "与えられた specific_criteria の各 item が手順に含まれる/満たすなら、その score を加点（合計5点で上限）。"
    )

    parts = []
    parts.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        parts.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            parts.append(f"- {it}")
    if sample.input.source_protocol_steps:
        parts.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            parts.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        parts.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            parts.append(f"- {fs}")
    if sample.input.references:
        parts.append("\n【参考文献】")
        for ref in sample.input.references:
            parts.append(f"- [{ref.id}] {ref.text}")

    parts.append("\n【生成された実験手順（Output）】")
    for s in steps:
        parts.append(f"- {s.id}. {s.text}")

    parts.append("\n【specific_criteria】")
    if sample.measurement and sample.measurement.specific_criteria:
        for item, sc in sample.measurement.specific_criteria.items():
            parts.append(f"- ({int(sc)}点) {item}")
    else:
        parts.append("- なし")

    user = "\n".join(parts)
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

def judge_with_llm(samples: List[ExampleSample], generated: list[dict]) -> pd.DataFrame:
    client = OpenAI(api_key=API_KEY) if 'API_KEY' in globals() and API_KEY else OpenAI()
    proc_map = {g['id']: [Step(id=it['id'], text=it['text']) for it in g['procedure_steps']] for g in generated}
    rows = []
    quota_exhausted = False
    def _is_insufficient_quota(err: Exception) -> bool:
        s = str(err)
        return 'insufficient_quota' in s or 'You exceeded your current quota' in s
    for sm in samples:
        if quota_exhausted:
            print(f"⏭️ スキップ採点: {sm.id}（クォータ不足）")
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'skipped_due_to_quota',
            })
            continue
        steps = proc_map.get(sm.id, [])
        msgs = build_judge_messages(sm, steps)
        try:
            completion = client.chat.completions.parse(
                model=JUDGE_MODEL,
                messages=msgs,
                temperature=JUDGE_TEMPERATURE,
                response_format=JudgeOutput,
            )
            parsed: JudgeOutput = completion.choices[0].message.parsed  # type: ignore
            rows.append({
                'id': sm.id,
                'general_score': parsed.general_score,
                'specific_score': parsed.specific_score,
                'total_score': parsed.final_score,
                'notes': parsed.notes or '',
            })
        except Exception as e:
            print(f"❌ 評価失敗: {sm.id}: {e}")
            if _is_insufficient_quota(e):
                print("⚠️ APIクォータ不足のため、以降の採点を中断します。プラン/課金設定をご確認ください。")
                quota_exhausted = True
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'evaluation_failed',
            })
    return pd.DataFrame(rows)

# 実行
df = judge_with_llm(samples, generated_results)
print(f"✅ LLM-as-a-judge: Scored {len(df)} samples (0-10)")
try:
    display(df[['id','general_score','specific_score','total_score']])
except Exception:
    print(df[['id','general_score','specific_score','total_score']])

csv_path = out_dir / f'eval_{JUDGE_MODEL}_{ts}.csv'
df.to_csv(csv_path, index=False, encoding="utf_8_sig")
print(f'📄 Saved: {csv_path}')

try:
    # ダウンロード確認ダイアログを出して、yならダウンロード
    print(f"Download file: {csv_path}")
    confirm = eval_js('confirm("生成されたCSVファイルをダウンロードしますか？")')
    if confirm:
      colab_files.download(str(csv_path))
    else:
      print("ダウンロードをスキップしました。")

except Exception:
    display(FileLink(str(csv_path.resolve())))


15:08:54 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:09:33 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:10:51 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:11:45 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:12:33 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:13:07 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:13:26 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:13:47 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:14:27 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:14:59 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ LLM-as-a-judge: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,private_test_1,5.0,5.0,10.0
1,private_test_10,4.0,2.0,6.0
2,private_test_2,4.0,1.0,5.0
3,private_test_3,5.0,4.0,9.0
4,private_test_4,5.0,5.0,10.0
5,private_test_5,3.0,5.0,8.0
6,private_test_6,5.0,5.0,10.0
7,private_test_7,5.0,5.0,10.0
8,private_test_8,4.0,3.0,7.0
9,private_test_9,4.0,4.0,8.0


📄 Saved: outputs/runs/eval_gpt-5-mini-2025-08-07_20251223_150826.csv
Download file: outputs/runs/eval_gpt-5-mini-2025-08-07_20251223_150826.csv


/Users/kato/Dropbox/Programs/gensurv/la-bench/outputs/runs/eval_gpt-5-mini-2025-08-07_20251223_150826.csv

In [8]:
# Cell 8: 評価用プロンプトとヘルパー
#@title 8. 評価用プロンプトとヘルパー { display-mode: "form" }

# 3種類の評価プロンプトと評価ヘルパー関数を定義します。
# Schema‑Strict Checklist Judge (Japanese)
STRICT_CHECKLIST_PROMPT = """あなたは生命科学実験の専門家であり、厳格な採点者です。与えられた Input（instruction, mandatory_objects, source_protocol_steps, expected_final_states, references）と、参加者の Output（procedure_steps）および問題ごとの specific_criteria を評価し、JudgeOutput のみを返してください。出力は以下のフィールドに完全準拠します：general_score, specific_score, final_score, general_reason, specific_matches, notes。余分なテキストは一切含めないでください。

<評価方針>
- 決定は二値（OK/NG）。不確実な場合は NG（無加点）。推測・補完はしない。
- 根拠は参加者 Output の記述のみから引用し、「」で囲む。Input は照合にのみ用い、引用しない。
- general_reason は必ず5行。「1)～5) 判定（OK/NG） | 根拠 | 引用」。OKの数はgeneral_scoreと一致させる。
- final_score = general_score + specific_score。整数。範囲 0–10。
- 過度の安全性の上限（cap）はgeneral_score算出後に適用し、notesに簡潔に記載。ない場合は空文字。

<手順>
1. Input から必須パラメータ・物品・手順論理・期待最終状態を抽出。
2. Output を走査し、操作可能な単位（µL/mL, mg/g 等）・器具明示・順序/分岐・計算整合性の有無を確認。
3. 5つの共通採点基準を独立に判定（OK/NG）。根拠は Output の具体フレーズを引用。
4. specific_criteria を満たす箇所のみ specific_matches に引用を記録し、該当点を加算。曖昧なら0。
5. 過度の安全性の条件に該当する場合、general_score を最大2に cap。notes に「cap理由」を短く記載。
6. JudgeOutput を返す。所定フィールド以外は出力しない。

# 共通採点基準（general_score、最大5点）
## 加点項目（各1点）
1. 「実験指示」におけるパラメータを、すべて誤りなく反映している。単なる再掲は認めない。
2. 「使用する物品」の要素が、すべて 誤りなく反映している。曖昧な総称のみは認めない。
3. 「元プロトコルの手順」の順序や分岐条件の論理構造を矛盾なく反映している。
4. 実行することで、「期待される最終状態」を得られる。論理的破綻や計算ミスがない。
5. 明示されていない部分を適切に補完している。記述が具体的である。

## 減点項目（各-1点）
1. 過度に不自然な日本語やハルシネーションを含む。
2. 計算ミスがある。
3. 手順に矛盾がある。

## 過度の安全性
以下のように過度に安全側に倒した回答の場合、共通採点基準（general_score）の合計点を最大2点とする。
- 入力の手順をそのまま出力として記載している。
- 出典の情報を適切な取捨選択なく記載している。
- 実験指示や元プロトコルに反して中間物を必要量の100倍用意するなど、みだりに実験条件を安全側に倒している。
### 適用例
- 採点結果が1点の場合 → 1点のまま
- 採点結果が3点の場合 → 2点に制限
- 採点結果が5点の場合 → 2点に制限

# 個別採点基準（specific_score、最大5点）
問題ごとに設定された採点項目に基づいて加点する。採点項目は以下のような観点で設定されている。
- 出題意図への適合性
- 安全性の考慮
- コスト効率
- 作業効率
- 実験精度向上への貢献
生成された実験手順が採点項目の条件を明示的に満たす場合のみ、該当する score を加点し、根拠となるフレーズをspecific_matchesに記録する。曖昧な記述は加点しない。

# 実験手順の詳細度に関する必須要件
実験手順（procedure_steps）は、単にプロトコルを構造化するだけではなく、当該分野の学部レベルのトレーニングを受けた人が読んで誤解なく実行できることを目指してください。そのために、以下の点を遵守してください。
- すべての条件が明示化されている。
- 不完全な記述が完成されている。
- 実行時に使用される具体的な数値が計算されている（µL/mL, mg/g など実際に操作できる単位。pmol, ng 等のみの提示は不可。換算して体積や質量を示した場合のみ可）。
- 操作対象や器具、装置など対象物が明示化されている。"""

# GPT‑5‑Tuned Agentic Judge (English)
GPT_5_TUNED_AGENTIC_JUDGE_PROMPT = """You are a life‑science expert and a strict grader. Evaluate the participant’s Output (procedure_steps) against the competition Input (instruction, mandatory_objects, source_protocol_steps, expected_final_states, references) and problem‑specific criteria. Return ONLY the JudgeOutput fields: general_score, specific_score, final_score, general_reason, specific_matches, notes. No extra text.

<persistence>
- Operate autonomously until evaluation is fully complete. Do NOT ask for clarification.
- Under uncertainty, bias to NG (no credit); proceed and document decisions.
</persistence>

<tool_preamble>
- Rephrase the goal briefly: “Score Output against Input using 5 general checks + specific criteria.”
- Plan: (1) Extract Input requirements; (2) parse Output; (3) apply five binary checks with quoted evidence; (4) apply over‑safety cap if triggered; (5) collect specific_matches; (6) compute scores; (7) emit JudgeOutput.
- Stop when: all five checks decided; specific_matches compiled; schema validated.
</tool_preamble>

<steering>
- reasoning_effort: medium (use minimal for latency‑sensitive runs).
- verbosity: low. No commentary beyond required fields.
- Determinism: low temperature; avoid emitting any fields beyond schema.
</steering>

<general_checks_definition>
1) Instruction parameters must be fully reflected AND operationalized (units/instruments). Restatement alone → NG.
2) Mandatory objects: ALL items concretely used; umbrella categories alone → NG.
3) Protocol logic: preserve order, branches, dependencies from source_protocol_steps without contradiction.
4) Achievability: steps logically yield expected_final_states; no calculation or logical errors.
5) Completeness/specificity: fill gaps appropriately; actionable units (µL/mL, mg/g) and instruments present. Non‑operational units alone (e.g., only ng/pmol) → NG unless converted.
</general_checks_definition>

<evidence_rules>
- Quote ONLY from Output for general_reason and specific_matches, wrapped in 「」.
- Use Input for alignment checks only; do not quote it.
- If Output lacks necessary evidence, mark NG; do not invent.
</evidence_rules>

<over_safety_cap>
Apply after computing general_score; cap to max 2 if:
- Output copies Input steps verbatim without operationalization.
- Output pastes references/external content without selection.
- Output inflates quantities (e.g., ×100 intermediates) contrary to Input/protocol without justification.
Record cap briefly in notes; otherwise notes is empty.
</over_safety_cap>

<format_strictness>
- Emit ONLY: general_score, specific_score, final_score, general_reason (exactly 5 lines “1)…5) 判定（OK/NG） | 根拠 | 引用”), specific_matches (list of quoted Output phrases), notes.
- Ensure OK count equals general_score; final_score equals sum.
</format_strictness>

# 共通採点基準（general_score、最大5点）
## 加点項目（各1点）
1. 「実験指示」におけるパラメータを、すべて誤りなく反映している。単なる再掲は認めない。
2. 「使用する物品」の要素が、すべて 誤りなく反映している。曖昧な総称のみは認めない。
3. 「元プロトコルの手順」の順序や分岐条件の論理構造を矛盾なく反映している。
4. 実行することで、「期待される最終状態」を得られる。論理的破綻や計算ミスがない。
5. 明示されていない部分を適切に補完している。記述が具体的である。

## 減点項目（各-1点）
1. 過度に不自然な日本語やハルシネーションを含む。
2. 計算ミスがある。
3. 手順に矛盾がある。

## 過度の安全性
以下のように過度に安全側に倒した回答の場合、共通採点基準（general_score）の合計点を最大2点とする。
- 入力の手順をそのまま出力として記載している。
- 出典の情報を適切な取捨選択なく記載している。
- 実験指示や元プロトコルに反して中間物を必要量の100倍用意するなど、みだりに実験条件を安全側に倒している。
### 適用例
- 採点結果が1点の場合 → 1点のまま
- 採点結果が3点の場合 → 2点に制限
- 採点結果が5点の場合 → 2点に制限

# 個別採点基準（specific_score、最大5点）
問題ごとに設定された採点項目に基づいて加点する。採点項目は以下のような観点で設定されている。
- 出題意図への適合性
- 安全性の考慮
- コスト効率
- 作業効率
- 実験精度向上への貢献
生成された実験手順が採点項目の条件を明示的に満たす場合のみ、該当する score を加点し、根拠となるフレーズをspecific_matchesに記録する。曖昧な記述は加点しない。

# 実験手順の詳細度に関する必須要件
実験手順（procedure_steps）は、単にプロトコルを構造化するだけではなく、当該分野の学部レベルのトレーニングを受けた人が読んで誤解なく実行できることを目指してください。そのために、以下の点を遵守してください。
- すべての条件が明示化されている。
- 不完全な記述が完成されている。
- 実行時に使用される具体的な数値が計算されている（µL/mL, mg/g など実際に操作できる単位。pmol, ng 等のみの提示は不可。換算して体積や質量を示した場合のみ可）。
- 操作対象や器具、装置など対象物が明示化されている。"""

# Contrastive Dual‑Pass Judge (Japanese)
CONTRASTIVE_DUAL_PASS_PROMPT = """あなたは生命科学分野の「対照型（コントラスト）デュアルパス」採点者です。Input（instruction, mandatory_objects, source_protocol_steps, expected_final_states, references）と参加者 Output（procedure_steps）、specific_criteria を評価し、JudgeOutput のみを返してください。余分な出力は厳禁です。

<persistence>
- 相談や確認は行わず、自律的に評価を完了する。
- 不確実な場合は NG（無加点）。推測はしない。判断理由は Output の引用で示す。
</persistence>

<評価フレーム>
- Pass A（Failure‑first）: 各共通基準について、NG となる根拠（欠落したパラメータ、未使用物品、順序/分岐の破綻、計算/論理エラー、操作不能な単位や器具の不在）を、Output からのみ抽出・引用。
- Pass B（Evidence‑for‑credit）: 各基準のOK条件を満たす直接的な証拠（操作可能な単位、具体的器具、全物品の使用、分岐維持、最終状態の達成）を Output からのみ抽出・引用。
- 衝突解決: Pass A が有効な失敗理由を示した場合、Pass B がそれを完全に打ち消す明確な引用証拠を示すときのみ OK。曖昧は NG。

<形式・算定>
- general_reason は必ず5行で「1)～5) 判定（OK/NG） | 根拠 | 引用」。引用は「」で囲む。
- specific_matches は、満たした specific_criteria ごとに Output の該当フレーズのみを列挙。曖昧なら追加しない。
- final_score = general_score + specific_score。過度の安全性 cap は general_score 算出後に適用し、notes に簡潔に記載。なければ空文字。
- 所定フィールド（general_score, specific_score, final_score, general_reason, specific_matches, notes）以外は出力禁止。

# 共通採点基準（general_score、最大5点）
## 加点項目（各1点）
1. 「実験指示」におけるパラメータを、すべて誤りなく反映している。単なる再掲は認めない。
2. 「使用する物品」の要素が、すべて 誤りなく反映している。曖昧な総称のみは認めない。
3. 「元プロトコルの手順」の順序や分岐条件の論理構造を矛盾なく反映している。
4. 実行することで、「期待される最終状態」を得られる。論理的破綻や計算ミスがない。
5. 明示されていない部分を適切に補完している。記述が具体的である。

## 減点項目（各-1点）
1. 過度に不自然な日本語やハルシネーションを含む。
2. 計算ミスがある。
3. 手順に矛盾がある。

## 過度の安全性
以下のように過度に安全側に倒した回答の場合、共通採点基準（general_score）の合計点を最大2点とする。
- 入力の手順をそのまま出力として記載している。
- 出典の情報を適切な取捨選択なく記載している。
- 実験指示や元プロトコルに反して中間物を必要量の100倍用意するなど、みだりに実験条件を安全側に倒している。
### 適用例
- 採点結果が1点の場合 → 1点のまま
- 採点結果が3点の場合 → 2点に制限
- 採点結果が5点の場合 → 2点に制限

# 個別採点基準（specific_score、最大5点）
問題ごとに設定された採点項目に基づいて加点する。採点項目は以下のような観点で設定されている。
- 出題意図への適合性
- 安全性の考慮
- コスト効率
- 作業効率
- 実験精度向上への貢献
生成された実験手順が採点項目の条件を明示的に満たす場合のみ、該当する score を加点し、根拠となるフレーズをspecific_matchesに記録する。曖昧な記述は加点しない。

# 実験手順の詳細度に関する必須要件
実験手順（procedure_steps）は、単にプロトコルを構造化するだけではなく、当該分野の学部レベルのトレーニングを受けた人が読んで誤解なく実行できることを目指してください。そのために、以下の点を遵守してください。
- すべての条件が明示化されている。
- 不完全な記述が完成されている。
- 実行時に使用される具体的な数値が計算されている（µL/mL, mg/g など実際に操作できる単位。pmol, ng 等のみの提示は不可。換算して体積や質量を示した場合のみ可）。
- 操作対象や器具、装置など対象物が明示化されている。"""

PROMPT_VARIANTS = {
    "strict_checklist": STRICT_CHECKLIST_PROMPT,
    "gpt5_tuned": GPT_5_TUNED_AGENTIC_JUDGE_PROMPT,
    "contrastive_dual_pass": CONTRASTIVE_DUAL_PASS_PROMPT,
}

PROMPT_VARIANT_LABELS = {
    "strict_checklist": "Prompt 1: 厳格チェックリスト",
    "gpt5_tuned": "Prompt 2: GPT-5-Tuned",
    "contrastive_dual_pass": "Prompt 3: 対照型デュアルパス",
}


def compose_judge_user_content(sample: ExampleSample, steps: List[Step]) -> str:
    parts: List[str] = []
    parts.append(f"# Input\n## 実験指示 (instruction)\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        parts.append("\n## 使用する物品 (mandatory_objects)")
        for it in sorted(sample.input.mandatory_objects):
            parts.append(f"- {it}")
    if sample.input.source_protocol_steps:
        parts.append("\n## 元プロトコルの手順（参考）(source_protocol_steps)")
        for st in sample.input.source_protocol_steps:
            parts.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        parts.append("\n## 期待される最終状態 (expected_final_states)")
        for fs in sorted(sample.input.expected_final_states):
            parts.append(f"- {fs}")
    if sample.input.references:
        parts.append("\n## 参考文献 (references)")
        for ref in sample.input.references:
            parts.append(f"- [{ref.id}] {ref.text}")
    parts.append("\n# 生成された実験手順（Output）")
    for stp in steps:
        parts.append(f"- {stp.id}. {stp.text}")
    parts.append("\n# specific_criteria")
    if sample.measurement and sample.measurement.specific_criteria:
        for item, sc in sample.measurement.specific_criteria.items():
            parts.append(f"- ({int(sc)}点): {item}")
    else:
        parts.append("- なし")
    return "\n".join(parts)

def build_judge_messages_with_prompt(prompt_key: str, sample: ExampleSample, steps: List[Step]) -> list[dict]:
    if prompt_key not in PROMPT_VARIANTS:
        raise ValueError(f"未知のプロンプトキーです: {prompt_key}")
    system = PROMPT_VARIANTS[prompt_key]
    user = compose_judge_user_content(sample, steps)
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

def ensure_evaluation_inputs_ready() -> None:
    missing = [name for name in ("samples", "generated_results", "out_dir", "ts") if name not in globals()]
    if missing:
        raise RuntimeError(f"評価に必要な変数が未定義です: {', '.join(missing)}")

def judge_with_prompt(prompt_key: str, samples: List[ExampleSample], generated: list[dict], *, model: str = JUDGE_MODEL, temperature: float = JUDGE_TEMPERATURE) -> pd.DataFrame:
    if prompt_key not in PROMPT_VARIANTS:
        raise ValueError(f"未知のプロンプトキーです: {prompt_key}")
    client = OpenAI(api_key=API_KEY) if 'API_KEY' in globals() and API_KEY else OpenAI()
    proc_map = {g['id']: [Step(id=it['id'], text=it['text']) for it in g.get('procedure_steps', [])] for g in generated}
    rows: List[dict] = []
    quota_exhausted = False

    def _is_insufficient_quota(err: Exception) -> bool:
        s = str(err).lower()
        return 'insufficient_quota' in s or 'exceeded your current quota' in s

    for sm in samples:
        if quota_exhausted:
            print(f"⏭️ スキップ採点({PROMPT_VARIANT_LABELS.get(prompt_key, prompt_key)}): {sm.id}（クォータ不足）")
            rows.append({
                'id': sm.id,
                'variant': prompt_key,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'general_reason': '',
                'specific_matches': [],
                'notes': 'skipped_due_to_quota',
            })
            continue
        steps = proc_map.get(sm.id, [])
        msgs = build_judge_messages_with_prompt(prompt_key, sm, steps)
        try:
            completion = client.chat.completions.parse(
                model=model,
                messages=msgs,
                temperature=temperature,
                response_format=JudgeOutput,
            )
            parsed: JudgeOutput = completion.choices[0].message.parsed  # type: ignore
            rows.append({
                'id': sm.id,
                'variant': prompt_key,
                'general_score': float(parsed.general_score),
                'specific_score': float(parsed.specific_score),
                'total_score': float(parsed.final_score),
                'general_reason': parsed.general_reason,
                'specific_matches': list(parsed.specific_matches),
                'notes': parsed.notes or '',
            })
        except Exception as e:
            label = PROMPT_VARIANT_LABELS.get(prompt_key, prompt_key)
            print(f"❌ {label} 評価失敗: {sm.id}: {e}")
            if _is_insufficient_quota(e):
                print("⚠️ APIクォータ不足のため、以降の採点を中断します。プラン/課金設定をご確認ください。")
                quota_exhausted = True
            rows.append({
                'id': sm.id,
                'variant': prompt_key,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'general_reason': '',
                'specific_matches': [],
                'notes': 'evaluation_failed',
            })
    df_variant = pd.DataFrame(rows)
    return df_variant


In [9]:
# Cell 9: Prompt 1 で評価
#@title 9. 評価: Prompt 1（strict_checklist） { display-mode: "form" }

ensure_evaluation_inputs_ready()
variant_key = "strict_checklist"
print(f"▶️ {PROMPT_VARIANT_LABELS[variant_key]} を実行します。")

df_strict_checklist = judge_with_prompt(variant_key, samples, generated_results)
print(f"✅ {PROMPT_VARIANT_LABELS[variant_key]}: Scored {len(df_strict_checklist)} samples (0-10)")
try:
    display(df_strict_checklist[['id', 'general_score', 'specific_score', 'total_score']])
except Exception:
    print(df_strict_checklist[['id', 'general_score', 'specific_score', 'total_score']])

csv_path_strict = out_dir / f'eval_{JUDGE_MODEL}_{variant_key}_{ts}.csv'
df_strict_checklist.to_csv(csv_path_strict, index=False, encoding="utf_8_sig")
print(f"📄 Saved: {csv_path_strict}")


▶️ Prompt 1: 厳格チェックリスト を実行します。


15:19:04 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:20:41 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:21:47 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:22:51 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:24:02 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:24:59 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:26:04 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:27:33 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:29:24 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:30:23 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Prompt 1: 厳格チェックリスト: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,private_test_1,5.0,5.0,10.0
1,private_test_10,3.0,4.0,7.0
2,private_test_2,4.0,0.0,4.0
3,private_test_3,4.0,4.0,8.0
4,private_test_4,2.0,4.0,6.0
5,private_test_5,4.0,4.0,8.0
6,private_test_6,4.0,4.0,8.0
7,private_test_7,5.0,5.0,10.0
8,private_test_8,1.0,5.0,6.0
9,private_test_9,4.0,2.0,6.0


📄 Saved: outputs/runs/eval_gpt-5-mini-2025-08-07_strict_checklist_20251223_150826.csv


In [10]:
# Cell 10: Prompt 2 で評価
#@title 10. 評価: Prompt 2（gpt5_tuned） { display-mode: "form" }

ensure_evaluation_inputs_ready()
variant_key = "gpt5_tuned"
print(f"▶️ {PROMPT_VARIANT_LABELS[variant_key]} を実行します。")

df_gpt5_tuned = judge_with_prompt(variant_key, samples, generated_results)
print(f"✅ {PROMPT_VARIANT_LABELS[variant_key]}: Scored {len(df_gpt5_tuned)} samples (0-10)")
try:
    display(df_gpt5_tuned[['id', 'general_score', 'specific_score', 'total_score']])
except Exception:
    print(df_gpt5_tuned[['id', 'general_score', 'specific_score', 'total_score']])

csv_path_gpt5 = out_dir / f'eval_{JUDGE_MODEL}_{variant_key}_{ts}.csv'
df_gpt5_tuned.to_csv(csv_path_gpt5, index=False, encoding="utf_8_sig")
print(f"📄 Saved: {csv_path_gpt5}")


▶️ Prompt 2: GPT-5-Tuned を実行します。


15:36:30 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:38:33 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:40:29 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:42:28 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:44:00 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:45:23 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:46:36 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:49:06 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:50:47 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:52:09 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Prompt 2: GPT-5-Tuned: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,private_test_1,5.0,5.0,10.0
1,private_test_10,4.0,2.0,6.0
2,private_test_2,3.0,0.0,3.0
3,private_test_3,4.0,4.0,8.0
4,private_test_4,5.0,5.0,10.0
5,private_test_5,4.0,4.0,8.0
6,private_test_6,4.0,4.0,8.0
7,private_test_7,5.0,5.0,10.0
8,private_test_8,4.0,2.0,6.0
9,private_test_9,5.0,3.0,8.0


📄 Saved: outputs/runs/eval_gpt-5-mini-2025-08-07_gpt5_tuned_20251223_150826.csv


In [11]:
# Cell 11: Prompt 3 で評価
#@title 11. 評価: Prompt 3（contrastive_dual_pass） { display-mode: "form" }

ensure_evaluation_inputs_ready()
variant_key = "contrastive_dual_pass"
print(f"▶️ {PROMPT_VARIANT_LABELS[variant_key]} を実行します。")

df_contrastive_dual_pass = judge_with_prompt(variant_key, samples, generated_results)
print(f"✅ {PROMPT_VARIANT_LABELS[variant_key]}: Scored {len(df_contrastive_dual_pass)} samples (0-10)")
try:
    display(df_contrastive_dual_pass[['id', 'general_score', 'specific_score', 'total_score']])
except Exception:
    print(df_contrastive_dual_pass[['id', 'general_score', 'specific_score', 'total_score']])

csv_path_contrastive = out_dir / f'eval_{JUDGE_MODEL}_{variant_key}_{ts}.csv'
df_contrastive_dual_pass.to_csv(csv_path_contrastive, index=False, encoding="utf_8_sig")
print(f"📄 Saved: {csv_path_contrastive}")


▶️ Prompt 3: 対照型デュアルパス を実行します。


15:53:10 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:54:56 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:56:23 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:57:46 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:58:56 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:00:30 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:02:29 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:03:51 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:05:11 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:07:24 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Prompt 3: 対照型デュアルパス: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,private_test_1,5.0,5.0,10.0
1,private_test_10,4.0,3.0,7.0
2,private_test_2,4.0,1.0,5.0
3,private_test_3,4.0,4.0,8.0
4,private_test_4,5.0,5.0,10.0
5,private_test_5,3.0,4.0,7.0
6,private_test_6,4.0,4.0,8.0
7,private_test_7,5.0,5.0,10.0
8,private_test_8,3.0,5.0,8.0
9,private_test_9,4.0,3.0,7.0


📄 Saved: outputs/runs/eval_gpt-5-mini-2025-08-07_contrastive_dual_pass_20251223_150826.csv


In [12]:
# Cell 12: 評価結果の比較
#@title 12. 新旧評価の比較 { display-mode: "form" }

required_frames = {
    'baseline(df)': 'df' in globals(),
    'strict_checklist': 'df_strict_checklist' in globals(),
    'gpt5_tuned': 'df_gpt5_tuned' in globals(),
    'contrastive_dual_pass': 'df_contrastive_dual_pass' in globals(),
}
missing = [name for name, available in required_frames.items() if not available]
if missing:
    raise RuntimeError(f"比較には次の結果が必要です: {', '.join(missing)}")

comparison = df[['id', 'total_score']].rename(columns={'total_score': 'baseline_total'})
variant_frames = {
    'strict_checklist_total': df_strict_checklist[['id', 'total_score']].rename(columns={'total_score': 'strict_checklist_total'}),
    'gpt5_tuned_total': df_gpt5_tuned[['id', 'total_score']].rename(columns={'total_score': 'gpt5_tuned_total'}),
    'contrastive_dual_pass_total': df_contrastive_dual_pass[['id', 'total_score']].rename(columns={'total_score': 'contrastive_dual_pass_total'}),
}
for _, frame in variant_frames.items():
    comparison = comparison.merge(frame, on='id', how='outer')

comparison['diff_strict_vs_baseline'] = comparison['strict_checklist_total'] - comparison['baseline_total']
comparison['diff_gpt5_vs_baseline'] = comparison['gpt5_tuned_total'] - comparison['baseline_total']
comparison['diff_contrastive_vs_baseline'] = comparison['contrastive_dual_pass_total'] - comparison['baseline_total']

score_cols = ['baseline_total', 'strict_checklist_total', 'gpt5_tuned_total', 'contrastive_dual_pass_total']
summary = comparison[score_cols].agg(['mean', 'std']).round(3)
print("📊 Score summary (mean/std):")
print(summary)

try:
    display(comparison)
except Exception:
    print(comparison)


📊 Score summary (mean/std):
      baseline_total  strict_checklist_total  gpt5_tuned_total  \
mean           8.300                   7.300             7.700   
std            1.829                   1.889             2.214   

      contrastive_dual_pass_total  
mean                        8.000  
std                         1.633  


,id,baseline_total,strict_checklist_total,gpt5_tuned_total,contrastive_dual_pass_total,diff_strict_vs_baseline,diff_gpt5_vs_baseline,diff_contrastive_vs_baseline
0,private_test_1,10.0,10.0,10.0,10.0,0.0,0.0,0.0
1,private_test_10,6.0,7.0,6.0,7.0,1.0,0.0,1.0
2,private_test_2,5.0,4.0,3.0,5.0,-1.0,-2.0,0.0
3,private_test_3,9.0,8.0,8.0,8.0,-1.0,-1.0,-1.0
4,private_test_4,10.0,6.0,10.0,10.0,-4.0,0.0,0.0
5,private_test_5,8.0,8.0,8.0,7.0,0.0,0.0,-1.0
6,private_test_6,10.0,8.0,8.0,8.0,-2.0,-2.0,-2.0
7,private_test_7,10.0,10.0,10.0,10.0,0.0,0.0,0.0
8,private_test_8,7.0,6.0,6.0,8.0,-1.0,-1.0,1.0
9,private_test_9,8.0,6.0,8.0,7.0,-2.0,0.0,-1.0
